In [1]:
from dotenv import load_dotenv
load_dotenv("../.env", override=True)

True

In [2]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("openai:gpt-4.1", temperature=0)

In [3]:
from langchain.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}' and content: {content}"

In [4]:
# Connect tools to a chat model
model_with_tools = llm.bind_tools([write_email], tool_choice="any", parallel_tool_calls=False)

In [99]:
from typing import Literal, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from email_assistant.utils import show_graph
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import ToolMessage, HumanMessage


def call_llm(state: MessagesState) -> MessagesState:
    """Run LLM"""

    output = model_with_tools.invoke(state["messages"])
    return {"messages": [output]}

def human_feedback(state: MessagesState) -> MessagesState:
    """Get human feedback before proceeding further"""
    feedback = interrupt("Please provide feedback: (Go Ahead/Describe Your Input)")
    messages = [
        HumanMessage(content=f"HUMAN FEEDBACK: {feedback}")
    ]
    return {"messages": messages}

def run_tool(state: MessagesState):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-2].tool_calls:
        observation = write_email.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result}

def human_feedback_evaluator(state: MessagesState) -> Literal["run_tool", "call_llm"]:
    """Route to tool handler, or Go back to LLM call"""
    
    # Get the last message
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the last message is a tool call, check if it's a Done tool call
    if 'GO AHEAD' in last_message.content.upper():
        return "run_tool"
    # Otherwise, we stop (reply to the user)
    # We are patching the last AIMessage from Tool_Call to Normal Message
    state["messages"][-2].content = "Email Body:" + state["messages"][-2].tool_calls[-1]['args']['content']
    state["messages"][-2].tool_calls = None
    state["messages"][-2].additional_kwargs = dict()
    return "call_llm"


workflow = StateGraph(MessagesState)
workflow.add_node("call_llm", call_llm)
workflow.add_node("run_tool", run_tool)
workflow.add_node("human_feedback", human_feedback)
workflow.add_edge(START, "call_llm")
workflow.add_edge("call_llm", "human_feedback")
workflow.add_conditional_edges("human_feedback", human_feedback_evaluator, {"run_tool": "run_tool", "call_llm": "call_llm", END: END})
workflow.add_edge("run_tool", END)


In [100]:
# Run the workflow

memory = InMemorySaver()
app = workflow.compile(checkpointer=memory)
# show_graph(app)

In [101]:


# Input
initial_input = {
    "messages": [
        {
            "role": "user"
            , "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!"
        }
    ]
}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in app.stream(initial_input, thread, stream_mode="updates"):
    if list(event.keys())[0] in ['call_llm', 'run_tool']:
        for m in event[list(event.keys())[0]]['messages']:
            m.pretty_print()
    elif list(event.keys())[0] == '__interrupt__':
        print("================================== Human Feedback Required ==================================")
        print(event['__interrupt__'][0].value)

================================== Ai Message ==================================
Tool Calls:
  write_email (call_iErDk2x1RASAzmyTHY7553cx)
 Call ID: call_iErDk2x1RASAzmyTHY7553cx
  Args:
    to: boss@company.ai
    subject: Confirmation: Attendance at Interrupt!
    content: Hi,

Thank you for letting me know about Interrupt! I would like to confirm that I am interested in attending the event.

Please let me know if there are any next steps or additional details I should be aware of.

Best regards,

[Your Name]
================================== Human Feedback Required ==================================
Please provide feedback: (Go Ahead/Describe Your Input)


In [102]:
for event in app.stream(Command(resume="Elaborate the mail body a little more"), thread, stream_mode="updates"):
    if list(event.keys())[0] in ['call_llm', 'run_tool']:
        for m in event[list(event.keys())[0]]['messages']:
            m.pretty_print()
    elif list(event.keys())[0] == '__interrupt__':
        print("================================== Human Feedback Required ==================================")
        print(event['__interrupt__'][0].value)

================================== Ai Message ==================================
Tool Calls:
  write_email (call_xiQ8lJMYPziDR7xzSegObe5N)
 Call ID: call_xiQ8lJMYPziDR7xzSegObe5N
  Args:
    to: boss@company.ai
    subject: Confirmation: Attendance at Interrupt!
    content: Hi,

Thank you for considering me for Interrupt! I am excited about the opportunity and would like to confirm my interest in attending the event. I believe it will be a great chance to learn, network, and bring back valuable insights to our team.

Please let me know if there are any preparations I should make or any specific sessions you recommend I focus on. I am looking forward to representing our team and making the most of this experience.

Thanks again for the opportunity.

Best regards,

[Your Name]
================================== Human Feedback Required ==================================
Please provide feedback: (Go Ahead/Describe Your Input)


In [104]:
for event in app.stream(Command(resume="Make the tone a little more professional"), thread, stream_mode="updates"):
    if list(event.keys())[0] in ['call_llm', 'run_tool']:
        for m in event[list(event.keys())[0]]['messages']:
            m.pretty_print()
    elif list(event.keys())[0] == '__interrupt__':
        print("================================== Human Feedback Required ==================================")
        print(event['__interrupt__'][0].value)

================================== Ai Message ==================================
Tool Calls:
  write_email (call_MZbQbGUPP1gAXNJxtMD8jr5U)
 Call ID: call_MZbQbGUPP1gAXNJxtMD8jr5U
  Args:
    to: boss@company.ai
    subject: Confirmation: Attendance at Interrupt!
    content: Dear [Boss's Name],

Thank you for considering me for the opportunity to attend Interrupt! I am writing to formally confirm my interest in participating in the event. I am confident that attending will provide valuable insights and learning opportunities that can benefit our team and contribute to our ongoing projects.

Please let me know if there are any specific sessions or topics you would like me to focus on, or if there are any preparations required ahead of the event. I look forward to representing our team and sharing key takeaways upon my return.

Thank you once again for this opportunity.

Best regards,

[Your Name]
================================== Human Feedback Required ================================

In [105]:
for event in app.stream(Command(resume="That looks good, Go Ahead"), thread, stream_mode="updates"):
    print(event)
    if list(event.keys())[0] in ['call_llm', 'run_tool']:
        for m in event[list(event.keys())[0]]['messages']:
            m.pretty_print()
    elif list(event.keys())[0] == '__interrupt__':
        print("================================== Human Feedback Required ==================================")
        print(event['__interrupt__'][0].value)

{'human_feedback': {'messages': [HumanMessage(content='HUMAN FEEDBACK: That looks good, Go Ahead', additional_kwargs={}, response_metadata={}, id='8fa809aa-e66a-4f56-b912-92bf2ec37473')]}}
{'run_tool': {'messages': [ToolMessage(content="Email sent to boss@company.ai with subject 'Confirmation: Attendance at Interrupt!' and content: Dear [Boss's Name],\n\nThank you for considering me for the opportunity to attend Interrupt! I am writing to formally confirm my interest in participating in the event. I am confident that attending will provide valuable insights and learning opportunities that can benefit our team and contribute to our ongoing projects.\n\nPlease let me know if there are any specific sessions or topics you would like me to focus on, or if there are any preparations required ahead of the event. I look forward to representing our team and sharing key takeaways upon my return.\n\nThank you once again for this opportunity.\n\nBest regards,\n\n[Your Name]", id='ab1c93ce-0eb0-44f